<a href="https://colab.research.google.com/github/Rishabh3693/stock-price-prediction-using-ml-lstm-xgboost-and-arima-/blob/main/Fake_news_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import TfidfVectorizer
from torch.utils.data import Dataset, DataLoader

In [ ]:
data = pd.read_csv("fake_or_real_news.csv")
data['label'] = data['label'].apply(lambda x: 0 if x == "REAL" else 1)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving fake_or_real_news.csv to fake_or_real_news.csv


In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data["text"], data["label"], test_size=0.2, random_state=42
)

In [ ]:
vectorizer = TfidfVectorizer(max_features=6000, stop_words="english")
X_train = vectorizer.fit_transform(train_texts).toarray()
X_test = vectorizer.transform(test_texts).toarray()

In [ ]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(train_labels.values, dtype=torch.long)
y_test_tensor = torch.tensor(test_labels.values, dtype=torch.long)

In [ ]:
class FakeNewsDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
batch_size = 32
train_dataset = FakeNewsDataset(X_train_tensor, y_train_tensor)
test_dataset = FakeNewsDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
class BiGRUClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(BiGRUClassifier, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)  # *2 for bidirectional
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = x.unsqueeze(1)  # Adding an extra dimension for sequence length
        gru_out, _ = self.gru(x)
        out = self.fc(gru_out[:, -1, :])  # Take last GRU output
        return self.softmax(out)

In [ ]:
input_size = 6000  # Same as TF-IDF vector size
hidden_size = 128
num_layers = 2
output_size = 2

model = BiGRUClassifier(input_size, hidden_size, num_layers, output_size)

# Define Loss and Optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")

Epoch [1/5], Loss: 0.3072
Epoch [2/5], Loss: 0.0646
Epoch [3/5], Loss: 0.0107
Epoch [4/5], Loss: 0.0014
Epoch [5/5], Loss: 0.0003


In [ ]:
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.9321
